In [1]:
import bldw
import glob
import numpy as np
import blimpy as bl
import matplotlib.pyplot as plt
import setigen
import os
import turbo_seti
%matplotlib inline

# for in-line (non-terminal) turboSETI
# as in Elan Lavie's notebook
import turbo_seti.find_doppler.seti_event as turbo
import turbo_seti.find_event as find
from turbo_seti.find_doppler.find_doppler import FindDoppler
from turbo_seti.find_event.find_event_pipeline import find_event_pipeline
from turbo_seti.find_event.plot_event_pipeline import plot_event_pipeline

In [36]:
DATADIR = '/datag/pipeline/AGBT19B_999_121/blc75_blp05/'
lfiles = ['blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0002.h5',
'blc75_guppi_58832_16530_HIP2792_0058.gpuspec.0002.h5',
'blc75_guppi_58832_16851_MESSIER031_0059.gpuspec.0002.h5',
'blc75_guppi_58832_17168_HIP3077_0060.gpuspec.0002.h5',
'blc75_guppi_58832_17485_MESSIER031_0061.gpuspec.0002.h5',
'blc75_guppi_58832_17801_HIP3223_0062.gpuspec.0002.h5']

In [37]:
# Uncomment for copying files into scratch directory — hopefully shouldn't be necessary for en-masse runs?

for file in lfiles:
    console = f'cp {DATADIR}{file} /datax/scratch/benjb/'
    os.system(console)

In [38]:
# remove DC spikes in absence of turboSETI blank_DC option
# not needed for time-resolution spectra

data_loc = '/datax/scratch/benjb/'

for i in range(len(lfiles)):
    fb = bl.Waterfall(DATADIR + lfiles[i])
    fb.blank_dc(64)
    fb.write_to_hdf5(f'{data_loc}bDC_{lfiles[i]}')

blimpy.waterfall INFO     Conversion time: 0.15sec
blimpy.waterfall INFO     Conversion time: 0.15sec
blimpy.waterfall INFO     Conversion time: 0.15sec
blimpy.waterfall INFO     Conversion time: 0.11sec
blimpy.waterfall INFO     Conversion time: 0.11sec
blimpy.waterfall INFO     Conversion time: 0.11sec


In [39]:
print('Beginning turboSETI search (FindDoppler version) ...')

data_loc = '/datax/scratch/benjb/'

for i in range(len(lfiles)):
    doppler = FindDoppler(data_loc + 'bDC_' + lfiles[i],
                      max_drift = 4,
                      snr = 2,       
                      out_dir = data_loc, # This is where the turboSETI output files will be stored.
                      n_coarse_chan = 64,
                      gpu_backend = True
                      #blank_dc = True
                     )
    doppler.search()

print('Done!')


Beginning turboSETI search (FindDoppler version) ...

turbo_seti version 2.0.18
blimpy version 2.0.11
h5py version 2.10.0

find_doppler.63 INFO     {'DIMENSION_LABELS': array([b'time', b'feed_id', b'frequency'], dtype=object), 'az_start': 0.0, 'data_type': 1, 'fch1': 1313.9634132385254, 'foff': -0.00286102294921875, 'ibeam': 1, 'machine_id': 20, 'nbeams': 1, 'nbits': 32, 'nchans': 65536, 'nifs': 1, 'rawdatafile': 'guppi_58832_16209_MESSIER031_0057.0000.raw', 'source_name': 'MESSIER031', 'src_dej': <Angle 41.2677 deg>, 'src_raj': <Angle 0.71234667 hourangle>, 'telescope_id': 6, 'tsamp': 1.0737418239999998, 'tstart': 58832.18760416667, 'za_start': 0.0}
find_doppler.63 INFO     File: /datax/scratch/benjb/bDC_blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0002.h5
 drift rates (min, max): (0.000000, 4.000000)
 SNR: 2.000000

Starting ET search using /datax/scratch/benjb/bDC_blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0002.h5
find_doppler.63 INFO     Parameters: datafile=/datax/scratch/

In [5]:
print('Beginning turboSETI search (command line version) ...')

for i in range(len(lfiles)):
    # Execute turboSETI in the terminal
    console = 'turboSETI ' + DATADIR + lfiles[i] + ' -M 4 -s 10 -o' + '/datax/scratch/benjb/'
    os.system(console)

print('Done!')

Beginning turboSETI search ...
Done!


In [5]:
print('Beginning plotSETI algorithm ...')

console = "plotSETI -f 3 -o " + DATADIR + " " + DATADIR
print(console)
os.system(console)

print('Done!')

Beginning plotSETI algorithm ...
plotSETI -f 3 -o /datax/scratch/benjb/m31_data/ /datax/scratch/benjb/m31_data/
Done!


In [ ]:
DATADIR = '/datag/pipeline/AGBT19B_999_121/blc75_blp05/'
lfiles = ['blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5',
'blc75_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5',
'blc75_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5',
'blc75_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5',
'blc75_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5',
'blc75_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5']

In [8]:
# Create a simple .lst file of the .h5 files in the data directory
h5_list = sorted(glob.glob(os.path.join(DATADIR, '*.h5')))
h5_list = [DATADIR + file for file in lfiles]
    
# This writes the .h5 files into a .lst, as required by the find_event_pipeline:
h5_list_path = os.path.join(data_loc,'h5_files.lst')
with open(h5_list_path, 'w') as f:
    for h5_path in h5_list:
        f.write(h5_path + '\n')

# You don't have to print, but it's a good way to check that your list is in the correct order:
with open(h5_list_path, 'r') as f:
    print(f.read())

/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_16209_MESSIER031_0057.gpuspec.0002.h5
/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_16530_HIP2792_0058.gpuspec.0002.h5
/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_16851_MESSIER031_0059.gpuspec.0002.h5
/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_17168_HIP3077_0060.gpuspec.0002.h5
/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_17485_MESSIER031_0061.gpuspec.0002.h5
/datag/pipeline/AGBT19B_999_121/blc72_blp02/blc72_guppi_58832_17801_HIP3223_0062.gpuspec.0002.h5



In [40]:
# Create a simple .lst file of the .dat files in the data directory
dat_list = sorted(glob.glob(os.path.join(data_loc, 'bDC_blc75*.dat')))
#dat_list = [data_loc + file[:-2] + 'dat' for file in lfiles]
    
# This writes the .dat files into a .lst, as required by the find_event_pipeline:
dat_list_path = os.path.join(data_loc, 'dat_files.lst')
with open(dat_list_path, 'w') as f:
    for dat_path in dat_list:
        f.write(dat_path + '\n')

# You don't have to print, but it's a good way to check that your list is in the correct order:
with open(dat_list_path, 'r') as f:
    print(f.read())

/datax/scratch/benjb/bDC_blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0002.dat
/datax/scratch/benjb/bDC_blc75_guppi_58832_16530_HIP2792_0058.gpuspec.0002.dat
/datax/scratch/benjb/bDC_blc75_guppi_58832_16851_MESSIER031_0059.gpuspec.0002.dat
/datax/scratch/benjb/bDC_blc75_guppi_58832_17168_HIP3077_0060.gpuspec.0002.dat
/datax/scratch/benjb/bDC_blc75_guppi_58832_17485_MESSIER031_0061.gpuspec.0002.dat
/datax/scratch/benjb/bDC_blc75_guppi_58832_17801_HIP3223_0062.gpuspec.0002.dat



In [43]:
csvf_path = os.path.join(data_loc, 'blc75_found_event_table_3.csv')
find_event_pipeline(dat_list_path, 
                    #h5_list_path,
                    SNR_cut=2,
                    check_zero_drift=True,
                    filter_threshold = 3, 
                    number_in_cadence = len(dat_list), 
                    csv_name=csvf_path, 
                    saving=True)


************   BEGINNING FIND_EVENT PIPELINE   **************

Assuming the first observation is an ON
find_event_pipeline: file = bDC_blc75_guppi_58832_16209_MESSIER031_0057.gpuspec.0002.dat, tstart = 58832.18760416667, source_name = MESSIER031
find_event_pipeline: file = bDC_blc75_guppi_58832_16530_HIP2792_0058.gpuspec.0002.dat, tstart = 58832.19131944444, source_name = HIP2792
find_event_pipeline: file = bDC_blc75_guppi_58832_16851_MESSIER031_0059.gpuspec.0002.dat, tstart = 58832.19503472222, source_name = MESSIER031
find_event_pipeline: file = bDC_blc75_guppi_58832_17168_HIP3077_0060.gpuspec.0002.dat, tstart = 58832.1987037037, source_name = HIP3077
find_event_pipeline: file = bDC_blc75_guppi_58832_17485_MESSIER031_0061.gpuspec.0002.dat, tstart = 58832.202372685184, source_name = MESSIER031
find_event_pipeline: file = bDC_blc75_guppi_58832_17801_HIP3223_0062.gpuspec.0002.dat, tstart = 58832.206030092595, source_name = HIP3223
There are 6 total files in the filelist /datax/scratch/

[]

In [12]:
# and finally we plot
plot_event_pipeline(csvf_path, # full path of the CSV file built by find_event_pipeline()
                    h5_list_path, # full path of text file containing the list of .h5 files
                    filter_spec='f{}'.format(2), # filter threshold
                    user_validation=False) # Non-interactive

plot_event_pipeline: Opened file /datax/scratch/benjb/m31_data/found_event_table.csv
plot_event_pipeline: file = blc71_guppi_58832_16209_MESSIER031_0057.gpuspec.0000.h5, tstart = 58832.18760416667, source_name = MESSIER031
plot_event_pipeline: file = blc71_guppi_58832_16530_HIP2792_0058.gpuspec.0000.h5, tstart = 58832.19131944444, source_name = HIP2792
plot_event_pipeline: file = blc71_guppi_58832_16851_MESSIER031_0059.gpuspec.0000.h5, tstart = 58832.19503472222, source_name = MESSIER031
plot_event_pipeline: file = blc71_guppi_58832_17168_HIP3077_0060.gpuspec.0000.h5, tstart = 58832.1987037037, source_name = HIP3077
plot_event_pipeline: file = blc71_guppi_58832_17485_MESSIER031_0061.gpuspec.0000.h5, tstart = 58832.202372685184, source_name = MESSIER031
plot_event_pipeline: file = blc71_guppi_58832_17801_HIP3223_0062.gpuspec.0000.h5, tstart = 58832.206030092595, source_name = HIP3223
Plotting some events for:  MESSIER031
There are 2 total events in the csv file /datax/scratch/benjb/m31_